In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn ibm-watson-machine-learning flask-ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.0 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.13.6-py3-none-any.whl size=77231 sha256=666bd192fe2bc33dbb919141a483f61bc3f2ebdea1844b8db699e2b8cb28a3d0
  Stored in directory: /root/.cache/pip/wheels/17/8c/76/f9472a53a2a80da414dd53ca55f08dbfe48ed6de76e51e3d5f
  Created wheel for ibm-cos-sdk-core: filename=ibm_co

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from ibm_watson_machine_learning import APIClient
import pickle


In [ ]:
# Load the dataset from the uploaded file
file_path = "/content/WA_Fn-UseC_-Telco-Customer-Churn.csv"  # Update path if needed
df = pd.read_csv(file_path)

# Display the first few rows
print(df.head())

# Check basic info
df.info()

# Check for missing values
print(df.isnull().sum())

# Summary statistics
print(df.describe())


   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

In [ ]:
# Fill missing numerical values with the mean
numerical_cols = df.select_dtypes(include=np.number).columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

# Fill missing categorical values with the mode
categorical_cols = df.select_dtypes(exclude=np.number).columns
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

In [ ]:
# Check for missing values
print(df.isnull().sum())

# Fill missing values with mean for numerical data only
for column in df.select_dtypes(include=['number']).columns: #Select only columns with numerical data types
    df[column].fillna(df[column].mean(), inplace=True) #Fill NA values for the selected column with the column's mean

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [ ]:
label_encoders = {}
categorical_cols = df.select_dtypes(include=["object"]).columns

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    print(f"Label Encoder for {col} saved.")
    # Display the updated DataFrame
print(df.head())

# Optionally, display the label encoders for each column
for col, encoder in label_encoders.items():
    print(f"Label Encoder for {col}:")
    print(encoder.classes_)  # Shows the mapping of original values to encoded values

   customerID  gender  SeniorCitizen  Partner  Dependents  tenure  \
0        5375       0              0        1           0       1   
1        3962       1              0        0           0      34   
2        2564       1              0        0           0       2   
3        5535       1              0        0           0      45   
4        6511       0              0        0           0       2   

   PhoneService  MultipleLines  InternetService  OnlineSecurity  ...  \
0             0              1                0               0  ...   
1             1              0                0               2  ...   
2             1              0                0               2  ...   
3             0              1                0               2  ...   
4             1              0                1               0  ...   

   DeviceProtection  TechSupport  StreamingTV  StreamingMovies  Contract  \
0                 0            0            0                0         0   


In [ ]:
scaler = StandardScaler()
numerical_cols = df.select_dtypes(include=["int64", "float64"]).columns
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
print("Scaler saved.")

Scaler saved.


In [ ]:
X = df.drop(columns=["Churn"])  # Drop target column
y = df["Churn"]  # Define target variable

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Data split into training and testing sets.")
print(df)

Data split into training and testing sets.
      customerID    gender  SeniorCitizen   Partner  Dependents    tenure  \
0       0.911890 -1.009559      -0.439916  1.034530   -0.654012 -1.277445   
1       0.216906  0.990532      -0.439916 -0.966622   -0.654012  0.066327   
2      -0.470701  0.990532      -0.439916 -0.966622   -0.654012 -1.236724   
3       0.990587  0.990532      -0.439916 -0.966622   -0.654012  0.514251   
4       1.470632 -1.009559      -0.439916 -0.966622   -0.654012 -1.236724   
...          ...       ...            ...       ...         ...       ...   
7038    0.655145  0.990532      -0.439916  1.034530    1.529024 -0.340876   
7039   -0.981733 -1.009559      -0.439916  1.034530    1.529024  1.613701   
7040   -0.075745 -1.009559      -0.439916  1.034530    1.529024 -0.870241   
7041    1.186835  0.990532       2.273159  1.034530   -0.654012 -1.155283   
7042   -0.636946  0.990532      -0.439916 -0.966622   -0.654012  1.369379   

      PhoneService  MultipleLine

In [ ]:
# Initialize and train the model
# Use RandomForestRegressor for continuous target variables
from sklearn.ensemble import RandomForestRegressor #Import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42) # Change to RandomForestRegressor
model.fit(X_train, y_train)

# Save model locally for later deployment
pickle.dump(model, open("churn_model.pkl", "wb"))

In [ ]:
# Predict on test data
y_pred = model.predict(X_test)

# Calculate R-squared (a common regression metric)
from sklearn.metrics import r2_score #Import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2:.4f}")

# You can also consider other regression metrics like Mean Squared Error (MSE) or Mean Absolute Error (MAE)
from sklearn.metrics import mean_squared_error, mean_absolute_error #Import MSE and MAE
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")

R-squared: 0.2710
Mean Squared Error: 0.7278
Mean Absolute Error: 0.6195


In [ ]:
wml_credentials = {
    "apikey": "fajcYKSClkw1E8nKFjbwyoKAPBZ5ZLyX6rpCsfQNOTrW",
    "url": "https://us-south.ml.cloud.ibm.com"
}

# Connect to WML
client = APIClient(wml_credentials)

# Set Deployment Space
client.set.default_space("19983994-8987-465b-90d8-40e2562538bb")  # Replace with your IBM Cloud Deployment Space ID


'SUCCESS'

In [ ]:
client.software_specifications.list()

----------------------------  ------------------------------------  ----  ------------  --------------------------
NAME                          ID                                    TYPE  STATE         REPLACEMENT
default_py3.6                 0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base  retired       runtime-24.1-py3.11
autoai-ts_rt23.1-py3.10       01ce9391-1a79-5a33-94fb-2e134337f314  base  constricted   autoai-ts_rt24.1-py3.11
kernel-spark3.2-scala2.12     020d69ce-7ac1-5e68-ac1a-31189867356a  base  retired
pytorch-onnx_1.3-py3.7-edt    069ea134-3346-5748-b513-49120e15d288  base  retired
tensorflow_rt23.1-py3.10      079a91e0-245f-5269-8926-3c20b28f37dc  base  constricted   tensorflow_rt24.1-py3.11
scikit-learn_0.20-py3.6       09c5a1d0-9c1e-4473-a344-eb7b665ff687  base  retired       runtime-24.1-py3.11
spark-mllib_3.0-scala_2.12    09f4cff0-90a7-5899-b9ed-1ef348aebdee  base  retired
pytorch-onnx_rt22.1-py3.9     0b848dd4-e681-5599-be41-b5f6fccc6471  base  retired       pytorch-on

,NAME,ID,TYPE,STATE,REPLACEMENT
0,default_py3.6,0062b8c9-8b7d-44a0-a9b9-46c416adcbd9,base,retired,runtime-24.1-py3.11
1,autoai-ts_rt23.1-py3.10,01ce9391-1a79-5a33-94fb-2e134337f314,base,constricted,autoai-ts_rt24.1-py3.11
2,kernel-spark3.2-scala2.12,020d69ce-7ac1-5e68-ac1a-31189867356a,base,retired,
3,pytorch-onnx_1.3-py3.7-edt,069ea134-3346-5748-b513-49120e15d288,base,retired,
4,tensorflow_rt23.1-py3.10,079a91e0-245f-5269-8926-3c20b28f37dc,base,constricted,tensorflow_rt24.1-py3.11
5,scikit-learn_0.20-py3.6,09c5a1d0-9c1e-4473-a344-eb7b665ff687,base,retired,runtime-24.1-py3.11
6,spark-mllib_3.0-scala_2.12,09f4cff0-90a7-5899-b9ed-1ef348aebdee,base,retired,
7,pytorch-onnx_rt22.1-py3.9,0b848dd4-e681-5599-be41-b5f6fccc6471,base,retired,pytorch-onnx_rt24.1-py3.11
8,ai-function_0.1-py3.6,0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda,base,retired,runtime-24.1-py3.11
9,watsonx-cfm-caikit-1.0,0cee3c55-472f-57b1-84bd-72f5d066dbe4,base,not_provided,


In [ ]:
software_spec_uid = client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
print("Software Specification UID:", software_spec_uid)

model_metadata = {
    client.repository.ModelMetaNames.NAME: "CustomerChurnDeployment",
    client.repository.ModelMetaNames.TYPE: "scikit-learn_1.0",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

Software Specification UID: 45f12dfe-aa78-5b8d-9f38-0ee223c47309


In [ ]:
import shutil

# Create a zip file containing the model
shutil.make_archive("/content/churn_model", 'zip', "/content", "churn_model.pkl")

'/content/churn_model.zip'

In [ ]:
import os
import shutil
import json

# Define the required directory for Watsonx
model_dir = "/content/model"
os.makedirs(model_dir, exist_ok=True)  # Create the 'model' folder if it doesn't exist

# Copy the model file into the 'model' directory
shutil.copy("/content/churn_model.pkl", model_dir)  # Ensure 'churn_model.pkl' is in /content/

# Define the metadata required for Watsonx
metadata = {
    "name": "Customer Churn Model",
    "description": "A machine learning model to predict customer churn.",
    "type": "scikit-learn_1.0",  # Make sure this matches your model framework
    "runtime": "python-3.10"
}

# Save metadata as JSON inside the 'model' folder
metadata_path = os.path.join(model_dir, "model_metadata.json")
with open(metadata_path, "w") as f:
    json.dump(metadata, f)

# Create a .zip file with the correct structure
shutil.make_archive("/content/CustomerChurnModel", 'zip', "/content", "model")

print("Zip file 'CustomerChurnModel.zip' created successfully!")

Zip file 'CustomerChurnModel.zip' created successfully!


In [ ]:
import zipfile

# List contents of the zip file
with zipfile.ZipFile("/content/CustomerChurnModel.zip", 'r') as zip_ref:
    zip_ref.printdir()

File Name                                             Modified             Size
model/                                         2025-02-20 06:13:54            0
model/model_metadata.json                      2025-02-20 06:13:54          156
model/churn_model.pkl                          2025-02-20 06:13:54     11557418


In [ ]:
from ibm_watson_machine_learning import APIClient

# IBM Cloud Credentials (Replace with your actual API key)
wml_credentials = {
    "apikey": "G7N6hMqTBR5BO0o0YjwgkbblwAiWW38GnsMUsmRihIKV",  # Use a new secure API key
    "url": "https://us-south.ml.cloud.ibm.com"
}

# Connect to IBM Watson ML
client = APIClient(wml_credentials)

# Set Deployment Space (Replace with your actual Space GUID)
space_id = "cce3afdc-7a71-45a7-a9f6-3d51184bbd59"  # Your Deployment Space GUID
client.set.default_space(space_id)

print("Successfully connected to IBM Watson Machine Learning!")

Successfully connected to IBM Watson Machine Learning!


In [ ]:
# Fetch all models in the deployment space
models = client.repository.get_details()

# Print full response to check the structure
import json
print(json.dumps(models, indent=4))  # Pretty-print the API response

{
    "models": {
        "resources": [
            {
                "entity": {
                    "software_spec": {
                        "id": "45f12dfe-aa78-5b8d-9f38-0ee223c47309"
                    },
                    "type": "scikit-learn_1.3"
                },
                "metadata": {
                    "created_at": "2025-02-15T15:11:26.549Z",
                    "description": "A machine learning  model to predict customer churn",
                    "id": "272f3a50-d452-49c2-8b15-125444bbf4dd",
                    "modified_at": "2025-02-15T15:11:30.926Z",
                    "name": "Customer Churn Model",
                    "owner": "IBMid-6940006W4S",
                    "space_id": "cce3afdc-7a71-45a7-a9f6-3d51184bbd59"
                },
                "system": {
                    "warnings": []
                }
            }
        ]
    },
    "experiments": {
        "resources": []
    },
    "pipeline": {
        "resources": []
    },
    "

In [ ]:
# Fetch model details
models = client.repository.get_details()

# Ensure 'resources' key exists before accessing it
if "resources" in models and models["resources"]:
    model_id = None
    model_name = "Customer Churn Model"  # Ensure name matches exactly

    for model in models["resources"]:
        if model["metadata"]["name"] == model_name:
            model_id = model["metadata"]["id"]
            break

    if model_id:
        print(" Model ID:", model_id)
    else:
        print(" Model not found in the deployment space.")
else:
    print(" No models found. Check if models exist in IBM Cloud.")

 No models found. Check if models exist in IBM Cloud.


In [ ]:
wml_credentials = {
    "apikey": "6X4Ku6ke_nnHpZldiy6nqEkyTVlS42c_-rH_HSJ5GgSz",
    "url": "https://us-south.ml.cloud.ibm.com"
}

In [ ]:
from ibm_watson_machine_learning import APIClient

try:
    # Authenticate with IBM Cloud
    client = APIClient(wml_credentials)
    print(" Successfully connected to IBM Watson Machine Learning!")
except Exception as e:
    print(" Authentication failed. Error:", str(e))

 Successfully connected to IBM Watson Machine Learning!


In [ ]:
from ibm_watson_machine_learning import APIClient

# IBM Cloud API Key (Replace with your actual key)
wml_credentials = {
    "apikey": "6X4Ku6ke_nnHpZldiy6nqEkyTVlS42c_-rH_HSJ5GgSz",
    "url": "https://us-south.ml.cloud.ibm.com"
}

# Connect to IBM Watson ML
client = APIClient(wml_credentials)

# Set Deployment Space
space_id = "cce3afdc-7a71-45a7-a9f6-3d51184bbd59"
client.set.default_space(space_id)

# Fetch Deployment ID
deployments = client.deployments.get_details()
deployment_id = None

for deployment in deployments.get('resources', []):
    if deployment["metadata"]["name"] == "Customer Churn Deployment":
        deployment_id = deployment["metadata"]["id"]
        break

if deployment_id:
    print("Deployment ID:", deployment_id)
else:
    print("Deployment not found. Ensure it’s created correctly.")

Deployment ID: 7fc085b2-6164-42c7-a36a-c8060be63ddb


In [ ]:
import requests

# IBM Cloud API Key (Replace with your actual API key)
api_key = "6X4Ku6ke_nnHpZldiy6nqEkyTVlS42c_-rH_HSJ5GgSz"

# IAM Authentication URL
auth_url = "https://iam.cloud.ibm.com/identity/token"

# Request IAM Token
auth_response = requests.post(
    auth_url,
    data={"grant_type": "urn:ibm:params:oauth:grant-type:apikey", "apikey": api_key},
    headers={"Content-Type": "application/x-www-form-urlencoded"},
)

# Extract IAM Token
iam_token = auth_response.json()["access_token"]
print("IAM Token generated successfully!")

IAM Token generated successfully!


In [ ]:
# Define API Endpoint (Use the public endpoint from Watsonx)
deployment_id = "7fc085b2-6164-42c7-a36a-c8060be63ddb"
deployment_endpoint = f"https://us-south.ml.cloud.ibm.com/ml/v4/deployments/{deployment_id}/predictions?version=2021-05-01"

# Define Headers with Updated IAM Token
headers = {
    "Authorization": f"Bearer {iam_token}",
    "Content-Type": "application/json"
}

# Example Input Data (Modify fields as per your model input)
payload = {
    "input_data": [
        {
            "fields": ["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges"],  # Adjust based on model input
            "values": [[0, 12, 79.85, 1290.10]]  # Example test case
        }
    ]
}

# Send API Request
response = requests.post(deployment_endpoint, json=payload, headers=headers)

# Print Prediction Response
print(" Prediction Response:", response.json())

 Prediction Response: {'trace': 'f8cc890045729f6b8ef54c4066f807e1', 'errors': [{'code': 'f8cc890045729f6b8ef54c4066f807e1', 'message': 'The feature names should match those that were passed during fit.\nFeature names seen at fit time, yet now missing:\n- Contract\n- Dependents\n- DeviceProtection\n- InternetService\n- MultipleLines\n- ...\n'}]}


In [ ]:
{
  "predictions": [
    {
      "fields": ["prediction", "confidence"],
      "values": [["No", 0.85]]
    }
  ]
}

{'predictions': [{'fields': ['prediction', 'confidence'],
   'values': [['No', 0.85]]}]}

In [ ]:
!pip install flask-ngrok flask pyngrok requests streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00


In [39]:
import logging
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import requests

logging.basicConfig(level=logging.INFO)
app = Flask(__name__)
run_with_ngrok(app)  # Automatically runs Ngrok when Flask starts


NGROK_AUTH_TOKEN = "2t7I3HEwJQ33BHNs82BS2QkOPCG_4CNYYC2DD5ww6ccPTefPw"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


IBM_CLOUD_API_KEY = "6X4Ku6ke_nnHpZldiy6nqEkyTVlS42c_-rH_HSJ5GgSz"
DEPLOYMENT_ID = "7fc085b2-6164-42c7-a36a-c8060be63ddb"
DEPLOYMENT_ENDPOINT = f"https://us-south.ml.cloud.ibm.com/ml/v4/deployments/{DEPLOYMENT_ID}/predictions?version=2021-05-01"


def get_iam_token():
    auth_url = "https://iam.cloud.ibm.com/identity/token"
    auth_response = requests.post(
        auth_url,
        data={"grant_type": "urn:ibm:params:oauth:grant-type:apikey", "apikey": IBM_CLOUD_API_KEY},
        headers={"Content-Type": "application/x-www-form-urlencoded"},
    )
    return auth_response.json()["access_token"]


public_url = ngrok.connect(5000).public_url
print(f" Flask is running at: {public_url}")


@app.route('/')
def home():
    return f"Welcome to the Customer Insights API!"


@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get JSON Data
        data = request.get_json()
        if not data:
            return jsonify({"error": "Invalid request. No data received."}), 400

        # Get IBM Watson ML IAM Token
        iam_token = get_iam_token()
        headers = {"Authorization": f"Bearer {iam_token}", "Content-Type": "application/json"}

        # Send request to IBM Watson ML
        response = requests.post(DEPLOYMENT_ENDPOINT, json=data, headers=headers)

        # Return IBM Watson Response
        return jsonify(response.json())

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run()

 Flask is running at: https://6c6f-34-75-197-209.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://6c6f-34-75-197-209.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [20/Feb/2025 07:12:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Feb/2025 07:12:37] "GET /favicon.ico HTTP/1.1" 404 -
